# 3. Modelo A

Modelo inicial:
- Baselines
- Grid search
- Voting ensemble

## Librerías

In [1]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score, \
roc_curve, roc_auc_score, ConfusionMatrixDisplay, multilabel_confusion_matrix

# Otros
import warnings
warnings.filterwarnings('ignore')

## Carga de datos

In [2]:
df = pd.read_csv(r'..\data\processed\train_3.csv')

## División de datos

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['nobeyesdad']),
                                                    df['nobeyesdad'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df['nobeyesdad'])

## Baselines

In [4]:
modelos = {LogisticRegression(random_state=42):'LogR',
           SVC(kernel='linear', random_state=42): 'SVC_linear',
           SVC(kernel='poly', degree=4): 'SVC_poly',
           SVC(random_state=42):'SVC_rbf',
           DecisionTreeClassifier(random_state=42):'DT',
           RandomForestClassifier(random_state=42, class_weight='balanced'):'RF',
           KNeighborsClassifier(n_neighbors=5):'KNEIGH',
           lgb.LGBMClassifier():'LGBM',
           XGBClassifier():'XGB'}

scores = ['accuracy']
data = []

for modelo in modelos:
    print(f'processing {modelo}')
    data.append([(cross_val_score(modelo, X_train, y_train, cv=5, scoring=score)).mean() for score in scores])

baselines = pd.DataFrame(data, columns=scores, index=modelos.values())
baselines.sort_values(by='accuracy', ascending=False)

processing LogisticRegression(random_state=42)
processing SVC(kernel='linear', random_state=42)
processing SVC(degree=4, kernel='poly')
processing SVC(random_state=42)
processing DecisionTreeClassifier(random_state=42)
processing RandomForestClassifier(class_weight='balanced', random_state=42)
processing KNeighborsClassifier()
processing LGBMClassifier()
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 782
[LightGBM] [Info] Number of data points in the train set: 13236, number of used features: 9
[LightGBM] [Info] Start training from score -2.104225
[LightGBM] [Info] Start training from score -1.903892
[LightGBM] [Info] Start training from score -2.148564
[LightGBM] [Info] Start training from score -2.125516
[LightGBM] [Info] Start training from score -1.962364
[LightGBM] [Info] Star

,accuracy
LGBM,0.888977
XGB,0.886680
RF,0.880999
SVC_poly,0.854286
SVC_rbf,0.845039
SVC_linear,0.839901
DT,0.829869
KNEIGH,0.784299
LogR,0.782909


Los 3 mejores modelos son **LightGBM**, **XGBoost** y **RandomForest**

## Grid Search

In [11]:
# LightGBM

LGBM_grid_1 = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_child_samples': [20, 30, 40],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

LGBM_grid_search_1 = GridSearchCV(lgb.LGBMClassifier(),
                           LGBM_grid_1,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1
                          )

LGBM_grid_search_1.fit(X_train, y_train)

In [ ]:
print(LGBM_grid_search_1.best_params_)

In [ ]:
LGBM_1 = lgb.LGBMClassifier(num_leaves=,
              learning_rate=,
              n_estimators=,
              max_depth=,
              min_child_samples=,
              subsample=,
              colsample_bytree=)

lgbm1_score = cross_val_score(LGBM_1, X_train, y_train, cv=5, scoring='accuracy').mean()
lgbm1_score

In [ ]:
LGBM_1.fit(X_train, y_train)
lgbm1_score_t = accuracy_score(y_test, LGBM_1.predict(X_test))
lgbm1_score_t

In [5]:
# XGBoost

XGB_grid_1 = {
       'n_estimators': [50, 100, 200],
       'learning_rate': [0.01, 0.05, 0.1],
       'max_depth': [3, 5, 7],
       'subsample': [0.6, 0.8, 1.0],
       'colsample_bytree': [0.6, 0.8, 1.0]
}

XGB_grid_search_1 = GridSearchCV(XGBClassifier(),
                           XGB_grid_1,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1
                          )

XGB_grid_search_1.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 200],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='accuracy')

In [6]:
print(XGB_grid_search_1.best_params_)

{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}


In [7]:
XGB_1 = XGBClassifier(n_estimators=100,
              learning_rate=0.1,
              max_depth=7,
              subsample=0.8,
              colsample_bytree=0.6)

xgb1_score = cross_val_score(XGB_1, X_train, y_train, cv=5, scoring='accuracy').mean()
xgb1_score

0.8934488290198205

In [ ]:
XGB_1.fit(X_train, y_train)
xgb1_score_t = accuracy_score(y_test, XGB_1.predict(X_test))
xgb1_score_t

In [8]:
# RandomForest

RF_grid_2 = {
    'n_estimators': [100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [8, 10, 12],
    'min_samples_leaf': [3, 4, 5],
    'bootstrap': [True]
}

RF_grid_search_2 = GridSearchCV(RandomForestClassifier(class_weight='balanced'),
                           RF_grid_2,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1
                          )

RF_grid_search_2.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(class_weight='balanced'),
             n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [5, 10, 15],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 150]},
             scoring='accuracy')

In [9]:
print(RF_grid_search_2.best_params_)

{'bootstrap': True, 'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 150}


In [10]:
RF_1 = RandomForestClassifier(n_estimators=150,
              max_depth=15,
              min_samples_split=8,
              min_samples_leaf=4,
              bootstrap=True)

rf1_score = cross_val_score(RF_1, X_train, y_train, cv=5, scoring='accuracy').mean()
rf1_score

0.8846252142148255

In [ ]:
RF_1.fit(X_train, y_train)
rf1_score_t = accuracy_score(y_test, RF_1.predict(X_test))
rf1_score_t

In [ ]:
# Score comparison on train

t1 = baselines.sort_values(by='accuracy', ascending=False).iloc[0:3]
t2 = pd.concat([t1, [lgbm1_score, xgb1_score, rf1_score]], axis=1)
t2

In [ ]:
# Score comparison on test

pd.DataFrame([lgbm1_score_t, xgb1_score_t, rf1_score_t], columns=['accuracy'], index=['LGBM','XGB','RF'])

## Ensemble

In [ ]:
voting_clf = VotingClassifier(estimators=[('lgbm', lgb.LGBMClassifier()),
                                          ('xgb', XGB_1),
                                          ('rf', RF_1)],
                            voting='soft',
                            verbose=False)

voting_clf.fit(X_train, y_train)

In [ ]:
cross_val_score(voting_clf, X_train, y_train, cv=5, scoring='accuracy').mean()

## Test validation

In [ ]:
accuracy_score(y_test, voting_clf.predict(X_test))

## Guardado

In [ ]:
import joblib

filename = 'my_model_1.sav'
joblib.dump(voting_clf, filename)